In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import ttest_ind
from sklearn.ensemble import VotingRegressor
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from scipy.stats import norm
from scipy.stats import ttest_ind
import joblib
import math

## Loading the Data

In [2]:
splitting_seed=42
np.random.seed(splitting_seed)
top_age=75# chaning this metric changes the upper bound of the target region
Num_models=25

#######
#Replace with proper addresses of sampled data. Data used here was seperated by E4-carriers and E4-NCS
#######
E4_AD_data_fp="C:/Users/woods/OneDrive/Documents/Research/Brain Lab/brain age paper/Sampled Data/both_apoe4_ad.csv"
NC_AD_data_fp="C:/Users/woods/OneDrive/Documents/Research/Brain Lab/brain age paper/Sampled Data/both_nonapoe4_ad.csv"
CU_data_fp="C:/Users/woods/OneDrive/Documents/Research/Brain Lab/brain age paper/Sampled Data/sampled_cn2.csv"

apoe4_ad=pd.read_csv(E4_AD_data_fp).fillna(0)
apoe4_ad['label']=pd.DataFrame(np.ones((apoe4_ad.shape[0])))
apoe4_ad['APOE4']=1 #adds genotype information

nonapoe4_ad=pd.read_csv(NC_AD_data_fp).fillna(0)
nonapoe4_ad['label']=pd.DataFrame(np.zeros((nonapoe4_ad.shape[0])))
nonapoe4_ad['APOE4']=0 #adds genotype information

data_cn=pd.read_csv(CU_data_fp)#already gentically labeled
#sort by genetic labels
apoe4_cn=data_cn[data_cn['APOE4']==1]
nonapoe4_cn=data_cn[data_cn['APOE4']==0] 


features = [ "CBTIA_1.0", "DEP2YRS_1.0", "DIABETES_1.0", "SOMATIC_1.0", "NACCBMI","INCONTU_1.0","EDUC",
          "B12DEF_2.0", "BPDIAS", "BPSYS", "HRATE", "SMOKYRS", "VISWCORR_1.0", "B12DEF_1.0", "NACCBETA_1.0", 
         "NACCLIPL_1.0", "NACCDBMD_1.0", "SEX",'APOE4']

brain = ["NACCAGE", "CEREALL", "CERECSF", "CEREGR", "CERETISS", "CEREWH", "CSFVOL", "FRCORT", "GRAYVOL", "HIPPOVOL", "LATVENT",
        "LCAC", "LCACM", "LCMF", "LCMFM", "LCUN", "LCUNM", "LENT", "LENTM", "LFRCORT", "LFUS", "LFUSM", "LHIPPO", "LINFPAR",
        "LINFPARM", "LINFTEMM", "LINFTEMP", "LINSULA","LINSULAM","LISTHC","LISTHCM","LLATOCC","LLATOCCM","LLATORBF",
        "LLATORBM","LLATVENT","LLING","LLINGM","LMEDORBF","LMEDORBM","LMIDTEMM","LMIDTEMP", "LOCCORT","LPARCEN","LPARCENM",
        "LPARCORT", "LPARHIP", "LPARHIPM", "LPARORB", "LPARORBM", "LPARSOP", "LPARSOPM", "LPARTRI", "LPARTRIM", "LPERCAL",
        "LPERCALM", "LPOSCEN", "LPOSCENM", "LPOSCIN", "LPOSCINM", "LPRECEN", "LPRECENM", "LPRECUN", "LPRECUNM", "LROSANC",
        "LROSANCM", "LROSMF", "LROSMFM", "LSUPFR", "LSUPFRM", "LSUPMAR", "LSUPMARM", "LSUPPAR", "LSUPPARM", "LSUPTEM", "LSUPTEMM",
        "LTEMPCOR", "LTRTEM", "LTRTEMM", "NACCBRNV", "NACCICV", "NACCWMVL", "OCCCORT", "PARCORT", "RCAC", "RCACM",
        "RCMF", "RCMFM", "RCUN", "RCUNM", "RENT", "RENTM", "RFRCORT", "RFUS", "RFUSM", "RHIPPO", "RINFPAR", "RINFPARM",
        "RINFTEMM", "RINFTEMP", "RINSULA", "RINSULAM", "RISTHC", "RISTHCM", "RLATOCC", "RLATOCCM", "RLATORBF", "RLATORBM",
        "RLATVENT", "RLING", "RLINGM", "RMEDORBF", "RMEDORBM", "RMIDTEMM", "RMIDTEMP", "ROCCORT", "RPARCEN", "RPARCENM",
        "RPARCORT", "RPARHIP", "RPARHIPM", "RPARORB", "RPARORBM", "RPARSOP", "RPARSOPM", "RPARTRI", "RPARTRIM", "RPERCAL",
        "RPERCALM", "RPOSCEN", "RPOSCENM", "RPOSCIN", "RPOSCINM", "RPRECEN", "RPRECENM", "RPRECUN", "RPRECUNM", "RROSANC",
        "RROSANCM", "RROSMF", "RROSMFM", "RSUPFR", "RSUPFRM", "RSUPMAR", "RSUPMARM", "RSUPPAR", "RSUPPARM", "RSUPTEM", 
        "RSUPTEMM", "RTEMPCOR", "RTRTEM", "RTRTEMM", "TEMPCOR", "THIRVENT", "WHITEVOL", "WMHVOL"]

all_features=brain+features

data_cn=data_cn[all_features]

########################Set data to features for APOE4 groups###########
apoe4_cn=apoe4_cn[all_features]
nonapoe4_cn=nonapoe4_cn[all_features]
apoe4_ad=apoe4_ad[all_features]
nonapoe4_ad=nonapoe4_ad[all_features]



########################Set data to features for Gender groups###########
data_cn=data_cn[all_features]
data_cn_MALE=data_cn[data_cn['SEX']==1]
data_cn_FEMALE=data_cn[data_cn['SEX']==2]
########################################################################

###########Seperate target variables #############
x_all_cu=data_cn.drop(["NACCAGE"], axis=1)
y_all_cu=data_cn.NACCAGE

x_apoe4_cu=apoe4_cn.drop(["NACCAGE"], axis=1)
y_apoe4_cu=apoe4_cn.NACCAGE
x_non_cu=nonapoe4_cn.drop(["NACCAGE"], axis=1)
y_non_cu=nonapoe4_cn.NACCAGE

data_apoe4_ad=apoe4_ad[apoe4_ad.NACCAGE<top_age]
data_non_ad=nonapoe4_ad[nonapoe4_ad.NACCAGE<top_age]
########################################################################################
ad_data_all=pd.concat([data_apoe4_ad,data_non_ad])#####################################

#########################################################################################
x_apoe4_ad=data_apoe4_ad.drop(["NACCAGE"], axis=1)
y_apoe4_ad=data_apoe4_ad.NACCAGE

x_non_ad=data_non_ad.drop(["NACCAGE"], axis=1)
y_non_ad=data_non_ad.NACCAGE

x_all_ad=ad_data_all.drop(["NACCAGE"],axis=1)
y_all_ad=ad_data_all.NACCAGE
####################################################


#######For generating gender seperated data#############

data_ad_MALE=ad_data_all[ad_data_all['SEX']==1]
data_ad_FEMALE=ad_data_all[ad_data_all['SEX']==2]

data_ad1=data_ad_MALE[data_ad_MALE.NACCAGE<top_age]#
data_ad2=data_ad_FEMALE[data_ad_FEMALE.NACCAGE<top_age]#

x_male=data_cn_MALE.drop(["NACCAGE"], axis=1)
y_male=data_cn_MALE.NACCAGE

x_female=data_cn_FEMALE.drop(["NACCAGE"], axis=1)
y_female=data_cn_FEMALE.NACCAGE

x_ad_male=data_ad1.drop(["NACCAGE"], axis=1)
y_ad_male=data_ad1.NACCAGE

x_ad_female=data_ad2.drop(["NACCAGE"], axis=1)
y_ad_female=data_ad2.NACCAGE
#########################################################

############## For groups D and E (APOE4 COMPARISON)#######################
target_length_D=round(len(x_apoe4_cu)/2)
target_length_E_E4=round(len(x_apoe4_cu))
target_length_E_NC=round(len(x_non_cu)/2+((len(x_non_cu)/2)-len(x_apoe4_cu)))#to correct for lack of CU APOE4 samples

indices_to_keep_D_e4=np.random.choice(len(x_apoe4_cu),target_length_D,replace=False)
indices_to_keep_D_NC=np.random.choice(len(x_non_cu),target_length_D,replace=False)

x_group_D_cu=pd.concat([x_apoe4_cu.iloc[indices_to_keep_D_e4],x_non_cu.iloc[indices_to_keep_D_NC]],axis=0)
y_group_D_cu=pd.concat([y_apoe4_cu.iloc[indices_to_keep_D_e4],y_non_cu.iloc[indices_to_keep_D_NC]],axis=0)

indices_to_keep_E_e4=np.random.choice(len(x_apoe4_cu),target_length_E_E4,replace=False)
indices_to_keep_E_NC=np.random.choice(len(x_non_cu),target_length_E_NC,replace=False)
x_group_E_cu=pd.concat([x_apoe4_cu.iloc[indices_to_keep_E_e4],x_non_cu.iloc[indices_to_keep_E_NC]],axis=0)
y_group_E_cu=pd.concat([y_apoe4_cu.iloc[indices_to_keep_E_e4],y_non_cu.iloc[indices_to_keep_E_NC]],axis=0)
###########################################################################

############## For groups D and E (APOE4 COMPARISON)#######################

target_length_E_E4=round(len(x_apoe4_cu))
target_length_E_NC=round(len(x_non_cu)/2+((len(x_non_cu)/2)-len(x_apoe4_cu)))#to correct for lack of CU APOE4 samples

indices_to_keep_D_e4=np.random.choice(len(x_apoe4_cu),target_length_D,replace=False)
indices_to_keep_D_NC=np.random.choice(len(x_non_cu),target_length_D,replace=False)
x_group_D_cu=pd.concat([x_apoe4_cu.iloc[indices_to_keep_D_e4],x_non_cu.iloc[indices_to_keep_D_NC]],axis=0)
y_group_D_cu=pd.concat([y_apoe4_cu.iloc[indices_to_keep_D_e4],y_non_cu.iloc[indices_to_keep_D_NC]],axis=0)

indices_to_keep_E_e4=np.random.choice(len(x_apoe4_cu),target_length_E_E4,replace=False)
indices_to_keep_E_NC=np.random.choice(len(x_non_cu),target_length_E_NC,replace=False)
x_group_E_cu=pd.concat([x_apoe4_cu.iloc[indices_to_keep_E_e4],x_non_cu.iloc[indices_to_keep_E_NC]],axis=0)
y_group_E_cu=pd.concat([y_apoe4_cu.iloc[indices_to_keep_E_e4],y_non_cu.iloc[indices_to_keep_E_NC]],axis=0)
###########################################################################








np.random.seed(42)#9999

APOE4 Specific Training


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Define the data structures used in cross validation, structures of _x subgroups are unnceassary and test on the opposite APOE4 genotype of the training group.
cv = KFold(n_splits=5, random_state=1, shuffle=True)
Num_models=25


split_indices_A = list(cv.split(x_apoe4_cu))
split_indices_B = list(cv.split(x_non_cu))
split_indices_C=list(cv.split(x_all_cu))
split_indices_D=list(cv.split(x_group_D_cu))
split_indices_E=list(cv.split(x_group_E_cu))

all_split_indices=[split_indices_A,split_indices_B,split_indices_C,split_indices_D,split_indices_E]
ave_result_cu_A = []
ave_result_ad_A = []

ave_result_cu_B = []
ave_result_ad_B = []

ave_result_cu_C_e4 = []
ave_result_ad_C_e4 = []
ave_result_cu_C_nc = []
ave_result_ad_C_nc = []

ave_result_cu_D_e4 = []
ave_result_cu_D_nc = []
ave_result_ad_D_e4 = []
ave_result_ad_D_nc = []

ave_result_cu_E_e4 = []
ave_result_cu_E_nc = []
ave_result_ad_E_e4 = []
ave_result_ad_E_nc = []

'''ave_result_cu_A_x = []
ave_result_cu_A_x = []
ave_result_ad_A_x = []
ave_result_ad_B_x = []'''

# Array of smooth predictions
ave_result_array = [
    ave_result_cu_A,
    ave_result_cu_B,
    ave_result_cu_C_e4,
    ave_result_cu_C_nc,
    ave_result_cu_D_e4,
    ave_result_cu_D_nc,
    ave_result_cu_E_e4,
    ave_result_cu_E_nc,
    ave_result_ad_A,
    ave_result_ad_B,
    ave_result_ad_C_e4,
    ave_result_ad_C_nc,
    ave_result_ad_D_e4,
    ave_result_ad_D_nc,
    ave_result_ad_E_e4,
    ave_result_ad_E_nc,

    '''ave_result_cu_A_x,
    ave_result_cu_A_x,
    ave_result_ad_A_x,
    ave_result_ad_B_x'''
]
####################################

smooth_rl_vals_cu_A = []
smooth_rl_vals_ad_A = []

smooth_rl_vals_cu_B = []
smooth_rl_vals_ad_B = []

smooth_rl_vals_cu_C_e4 = []
smooth_rl_vals_ad_C_e4 = []
smooth_rl_vals_cu_C_nc = []
smooth_rl_vals_ad_C_nc = []

smooth_rl_vals_cu_D_e4 = []
smooth_rl_vals_cu_D_nc = []
smooth_rl_vals_ad_D_e4 = []
smooth_rl_vals_ad_D_nc = []

smooth_rl_vals_cu_E_e4 = []
smooth_rl_vals_cu_E_nc = []
smooth_rl_vals_ad_E_e4 = []
smooth_rl_vals_ad_E_nc = []

smooth_rl_vals_cu_A_e4 = []
smooth_rl_vals_ad_A_nc = []
smooth_rl_vals_cu_B_e4 = []
smooth_rl_vals_ad_B_nc = []

'''smooth_rl_vals_cu_A_x=[]
smooth_rl_vals_cu_B_x=[]
smooth_rl_vals_ad_A_x=[]
smooth_rl_vals_ad_B_x=[]'''


smooth_rl_vals_array = [
    smooth_rl_vals_cu_A,
    smooth_rl_vals_cu_B,
    smooth_rl_vals_cu_C_e4,
    smooth_rl_vals_cu_C_nc,
    smooth_rl_vals_cu_D_e4,
    smooth_rl_vals_cu_D_nc,
    smooth_rl_vals_cu_E_e4,
    smooth_rl_vals_cu_E_nc,
    smooth_rl_vals_ad_A,
    smooth_rl_vals_ad_B,
    smooth_rl_vals_ad_C_e4,
    smooth_rl_vals_ad_C_nc,
    smooth_rl_vals_ad_D_e4,
    smooth_rl_vals_ad_D_nc,
    smooth_rl_vals_ad_E_e4,
    smooth_rl_vals_ad_E_nc,
    '''smooth_rl_vals_cu_A_x,
    smooth_rl_vals_cu_B_x,
    smooth_rl_vals_ad_A_x,
    smooth_rl_vals_ad_B_x,'''


]

##################################
##################################

smooth_preds_cu_A = []
smooth_preds_ad_A = []

smooth_preds_cu_B = []
smooth_preds_ad_B = []

smooth_preds_cu_C_e4 = []
smooth_preds_ad_C_e4 = []
smooth_preds_cu_C_nc = []
smooth_preds_ad_C_nc = []

smooth_preds_cu_D_e4 = []
smooth_preds_cu_D_nc = []
smooth_preds_ad_D_e4 = []
smooth_preds_ad_D_nc = []

smooth_preds_cu_E_e4 = []
smooth_preds_cu_E_nc = []
smooth_preds_ad_E_e4 = []
smooth_preds_ad_E_nc = []

'''smooth_preds_cu_A_x = []
smooth_preds_cu_B_x = []
smooth_preds_ad_A_x = []
smooth_preds_ad_B_x = []'''

# Array of smooth predictions, change size to 20 if including_x groups
smooth_preds_array = np.empty(20, dtype=object)

###############################
###############################

smooth_preds_cu_list_A = []
smooth_preds_cu_list_B = []
smooth_preds_cu_list_C_e4 = []
smooth_preds_cu_list_C_nc = []
smooth_preds_cu_list_D_e4 = []
smooth_preds_cu_list_D_NC = []
smooth_preds_cu_list_E_e4 = []
smooth_preds_cu_list_E_NC = []

smooth_preds_ad_list_A = []
smooth_preds_ad_list_B = []
smooth_preds_ad_list_C_e4 = []
smooth_preds_ad_list_C_nc = []
smooth_preds_ad_list_D_e4 = []
smooth_preds_ad_list_D_NC = []
smooth_preds_ad_list_E_e4 = []
smooth_preds_ad_list_E_NC = []

'''smooth_preds_cu_list_A_x = []
smooth_preds_ad_list_A_x = []
smooth_preds_cu_list_B_x = []
smooth_preds_ad_list_B_x = []'''

# Array of smooth predictions lists

ensemble_predictions_cu_A = []
ensemble_predictions_ad_A = []

ensemble_predictions_cu_B = []
ensemble_predictions_ad_B = []

ensemble_predictions_cu_C_e4 = []
ensemble_predictions_ad_C_e4 = []
ensemble_predictions_cu_C_nc = []
ensemble_predictions_ad_C_nc = []

ensemble_predictions_cu_D_e4 = []
ensemble_predictions_cu_D_nc = []
ensemble_predictions_ad_D_e4 = []
ensemble_predictions_ad_D_nc = []

ensemble_predictions_cu_E_e4 = []
ensemble_predictions_cu_E_nc = []
ensemble_predictions_ad_E_e4 = []
ensemble_predictions_ad_E_nc = []
'''ensemble_predictions_cu_A_x = []
ensemble_predictions_ad_A_x = []
ensemble_predictions_cu_A_x = []
ensemble_predictions_ad_B_x = []'''


# Array of smooth predictions lists
smooth_preds_lists_array = np.empty((Num_models,20),dtype=object)
################################
for i in range(Num_models):
    ################################

    ###################################

    ##################################
    ensemble_predictions_cu_A = []
    ensemble_predictions_ad_A = []

    ensemble_predictions_cu_B = []
    ensemble_predictions_ad_B = []

    ensemble_predictions_cu_C_e4 = []
    ensemble_predictions_ad_C_e4 = []
    ensemble_predictions_cu_C_nc = []
    ensemble_predictions_ad_C_nc = []

    ensemble_predictions_cu_D_e4 = []
    ensemble_predictions_cu_D_nc = []
    ensemble_predictions_ad_D_e4 = []
    ensemble_predictions_ad_D_nc = []

    ensemble_predictions_cu_E_e4 = []
    ensemble_predictions_cu_E_nc = []
    ensemble_predictions_ad_E_e4 = []
    ensemble_predictions_ad_E_nc = []

    '''ensemble_predictions_cu_A_x = []
    ensemble_predictions_cu_B_x = []
    ensemble_predictions_ad_A_x = []
    ensemble_predictions_ad_B_x = []'''
    # Define real values lists for each category

    real_vals_cu_A = []
    real_vals_cu_B = []
    real_vals_ad_A = []
    real_vals_ad_B = []

    real_vals_cu_C_e4 = []
    real_vals_cu_C_NC = []
    real_vals_ad_C_e4 = []
    real_vals_ad_C_NC = []

    real_vals_cu_D_e4 = []
    real_vals_cu_D_NC = []
    real_vals_ad_D_e4 = []
    real_vals_ad_D_NC = []

    real_vals_cu_E_e4 = []
    real_vals_cu_E_NC = []
    real_vals_ad_E_e4 = []
    real_vals_ad_E_NC = []

    '''real_vals_cu_A_x = []
    real_vals_cu_B_x = []
    real_vals_ad_A_x = []
    real_vals_ad_B_x = []'''

    # Array of real values lists

#################################### Training sector #######################################################################
    model=RandomForestRegressor(n_estimators=800, min_samples_split=10, min_samples_leaf=2, max_features='sqrt', max_depth=30)

    for split_group_num, split_indices in enumerate(all_split_indices):
        for fold_num, (train_index, test_index) in enumerate(split_indices):
            print(split_group_num)
            if split_group_num==0:#A
                x_cn=x_apoe4_cu
                y_cn=y_apoe4_cu
                x_ad=x_apoe4_ad
            if split_group_num==1:#B
                x_cn=x_non_cu
                y_cn=y_non_cu
                x_ad=x_non_ad
            if split_group_num==2:#C
                x_cn=x_all_cu
                y_cn=y_all_cu

                x_ad_e4=x_apoe4_cu
                x_ad_nc=x_apoe4_ad
            if split_group_num==3:#D
                x_cn=x_group_D_cu
                y_cn=y_group_D_cu
                x_ad=x_apoe4_cu
            if split_group_num==4:#E
                x_cn=x_group_E_cu
                y_cn=y_group_E_cu
                x_ad=x_non_ad
            x_cn = pd.DataFrame(x_cn)
            y_cn = pd.DataFrame(y_cn)
            x_ad = pd.DataFrame(x_ad)
            y_all_ad=pd.DataFrame(y_all_ad)
            print("split group num=",split_group_num)

            x_train, x_test = x_cn.iloc[train_index], x_cn.iloc[test_index]
            y_train, y_test = y_cn.iloc[train_index], y_cn.iloc[test_index]


  



 
            print('model fitting')
            print('length of x_train=',len(x_train))
            print('length of x_train apoe4=',len(x_train[x_train['APOE4']==1]))
            [x_test['APOE4']==1]
            print(len(x_train))
            print(len(x_train))
            model.fit(x_train, y_train)
            print('fitting complete')
###############################################################################################################
            x_cn = pd.DataFrame(x_cn)
            y_cn = pd.DataFrame(y_cn)
            x_ad = pd.DataFrame(x_ad)
############################################ Testing for each groups      #####################################
            if (split_group_num==0):# Group A
                y_pred_cu_A=model.predict(x_test)
                y_pred_ad_A=model.predict(x_apoe4_ad)
                ensemble_predictions_cu_A.append(y_pred_cu_A)
                ensemble_predictions_ad_A.append(y_pred_ad_A)
                real_vals_cu_A.append(y_test)
                real_vals_ad_A.append(y_all_ad[x_all_ad['APOE4']==1])

                '''y_pred_cu_A_x=model.predict(x_non_cu)
                y_pred_ad_A_x=model.predict(x_non_ad)
                ensemble_predictions_cu_A_x.append(y_pred_cu_A_x)
                ensemble_predictions_ad_A_x.append(y_pred_ad_A_x)
                real_vals_cu_A_x.append(y_non_cu)
                real_vals_ad_A_x.append(y_all_ad[x_all_ad['APOE4']==0])'''

            if (split_group_num==1):#Group B
                y_pred_cu_B=model.predict(x_test)
                y_pred_ad_B=model.predict(x_non_ad)
                ensemble_predictions_cu_B.append(y_pred_cu_B)
                ensemble_predictions_ad_B.append(y_pred_ad_B)
                real_vals_cu_B.append(y_test)
                real_vals_ad_B.append(y_all_ad[x_all_ad['APOE4']==0])

                '''y_pred_cu_B_x=model.predict(x_apoe4_cu)
                y_pred_ad_B_x=model.predict(x_apoe4_ad)
                ensemble_predictions_cu_B_x.append(y_pred_cu_B_x)
                ensemble_predictions_ad_B_x.append(y_pred_ad_B_x)
                real_vals_cu_B_x.append(y_apoe4_cu)
                real_vals_ad_B_x.append(y_all_ad[x_all_ad['APOE4']==1])'''
            
            if (split_group_num==2):#Group C
                y_pred_cu_e4_C=model.predict(x_test[x_test['APOE4']==1])
                y_pred_ad_e4_C=model.predict(x_all_ad[x_all_ad['APOE4']==1])
                ensemble_predictions_cu_C_e4.append(y_pred_cu_e4_C)
                ensemble_predictions_ad_C_e4.append(y_pred_ad_e4_C)
                real_vals_cu_C_e4.append(y_test[x_test['APOE4']==1])
                real_vals_ad_C_e4.append(y_all_ad[x_all_ad['APOE4']==1])

                y_pred_cu_nc_C=model.predict(x_test[x_test['APOE4']==0])
                y_pred_ad_nc_C=model.predict(x_all_ad[x_all_ad['APOE4']==0])
                ensemble_predictions_cu_C_nc.append(y_pred_cu_nc_C)
                ensemble_predictions_ad_C_nc.append(y_pred_ad_nc_C)
                real_vals_cu_C_NC.append(y_test[x_test['APOE4']==0])
                real_vals_ad_C_NC.append(y_all_ad[x_all_ad['APOE4']==0])

            if (split_group_num==3):#Group D
                y_pred_cu_D_e4=model.predict(x_test[x_test['APOE4']==1])
                y_pred_ad_D_e4=model.predict(x_all_ad[x_all_ad['APOE4']==1])
                ensemble_predictions_cu_D_e4.append(y_pred_cu_D_e4)
                ensemble_predictions_ad_D_e4.append(y_pred_ad_D_e4)
                real_vals_cu_D_e4.append(y_test[x_test['APOE4']==1])
                real_vals_ad_D_e4.append(y_all_ad[x_all_ad['APOE4']==1])

                y_pred_cu_D_nc=model.predict(x_test[x_test['APOE4']==0])
                y_pred_ad_D_nc=model.predict(x_all_ad[x_all_ad['APOE4']==0])
                ensemble_predictions_cu_D_nc.append(y_pred_cu_D_nc)
                ensemble_predictions_ad_D_nc.append(y_pred_ad_D_nc)
                real_vals_cu_D_NC.append(y_test[x_test['APOE4']==0])
                real_vals_ad_D_NC.append(y_all_ad[x_all_ad['APOE4']==0])

            if (split_group_num==4):#Group #
                y_pred_cu_E_e4=model.predict(x_test[x_test['APOE4']==1])
                y_pred_ad_E_e4=model.predict(x_all_ad[x_all_ad['APOE4']==1])
                ensemble_predictions_cu_E_e4.append(y_pred_cu_E_e4)
                ensemble_predictions_ad_E_e4.append(y_pred_ad_E_e4)
                real_vals_cu_E_e4.append(y_test[x_test['APOE4']==1])
                real_vals_ad_E_e4.append(y_all_ad[x_all_ad['APOE4']==1])

                y_pred_cu_E_nc=model.predict(x_test[x_test['APOE4']==0])
                y_pred_ad_E_nc=model.predict(x_all_ad[x_all_ad['APOE4']==0])
                ensemble_predictions_cu_E_nc.append(y_pred_cu_E_nc)
                ensemble_predictions_ad_E_nc.append(y_pred_ad_E_nc)
                real_vals_cu_E_NC.append(y_test[x_test['APOE4']==0])
                real_vals_ad_E_NC.append(y_all_ad[x_all_ad['APOE4']==0])
##############################################################################################################
    ensembled_preds_array = [
    ensemble_predictions_cu_A,
    ensemble_predictions_cu_B,
    ensemble_predictions_cu_C_e4,
    ensemble_predictions_cu_C_nc,
    ensemble_predictions_cu_D_e4,   # New term
    ensemble_predictions_cu_D_nc,   # New term
    ensemble_predictions_cu_E_e4,   # New term
    ensemble_predictions_cu_E_nc,   # New term
    ensemble_predictions_ad_A,
    ensemble_predictions_ad_B,
    ensemble_predictions_ad_C_e4,
    ensemble_predictions_ad_C_nc,
    ensemble_predictions_ad_D_e4, 
    ensemble_predictions_ad_D_nc,   
    ensemble_predictions_ad_E_e4,   
    ensemble_predictions_ad_E_nc,

    '''ensemble_predictions_cu_A_x,
    ensemble_predictions_cu_B_x,
    ensemble_predictions_ad_A_x,
    ensemble_predictions_ad_B_x'''
    
    ]  

    real_vals_array = [
    real_vals_cu_A,
    real_vals_cu_B,
    real_vals_cu_C_e4,
    real_vals_cu_C_NC,
    real_vals_cu_D_e4,
    real_vals_cu_D_NC,
    real_vals_cu_E_e4,
    real_vals_cu_E_NC,
    real_vals_ad_A,
    real_vals_ad_B,
    real_vals_ad_C_e4,
    real_vals_ad_C_NC,
    real_vals_ad_D_e4,
    real_vals_ad_D_NC,
    real_vals_ad_E_e4,
    real_vals_ad_E_NC,

'''    real_vals_cu_A_x,
    real_vals_cu_B_x,
    real_vals_ad_A_x,
    real_vals_ad_B_x'''
    

]


############################################ Ensembling Portion ##############################################

    for j in range(20):#range is jsut size of all arrays.
        print(j)
        print(len(real_vals_array[j]))
        smooth_rl_vals_array[j]=np.concatenate(real_vals_array[j])
######################error above
        if j<8:
    
            smooth_preds_array[j]=np.concatenate(ensembled_preds_array[j])
        if j>=8:
            smooth_preds_array[j]=np.mean(ensembled_preds_array[j],axis=0)
        smooth_preds_lists_array[i][j]=(np.array(smooth_preds_array[j]))
ave_result_array=np.mean(smooth_preds_lists_array,axis=0)



Prepare data for further analysis

In [6]:
for i in range(len(smooth_rl_vals_array)):
    hold=smooth_rl_vals_array[i].flatten()
    smooth_rl_vals_array[i]=hold

smooth_rl_vals_cu_A = smooth_rl_vals_array[0]
smooth_rl_vals_cu_B = smooth_rl_vals_array[1]

smooth_rl_vals_cu_C_e4 = smooth_rl_vals_array[2]
smooth_rl_vals_cu_C_nc = smooth_rl_vals_array[3]

smooth_rl_vals_cu_D_e4 = smooth_rl_vals_array[4]
smooth_rl_vals_cu_D_nc = smooth_rl_vals_array[5]


smooth_rl_vals_cu_E_e4 = smooth_rl_vals_array[6]
smooth_rl_vals_cu_E_nc = smooth_rl_vals_array[7]



smooth_rl_vals_ad_A = smooth_rl_vals_array[8][:int(len(smooth_rl_vals_array[8]) / 5)]
smooth_rl_vals_ad_B = smooth_rl_vals_array[9][:int(len(smooth_rl_vals_array[9]) / 5)]

smooth_rl_vals_ad_C_e4 = smooth_rl_vals_array[10][:int(len(smooth_rl_vals_array[10]) / 5)]
smooth_rl_vals_ad_C_nc = smooth_rl_vals_array[11][:int(len(smooth_rl_vals_array[11]) / 5)]

smooth_rl_vals_ad_D_e4 = smooth_rl_vals_array[12][:int(len(smooth_rl_vals_array[12]) / 5)]
smooth_rl_vals_ad_D_nc = smooth_rl_vals_array[13][:int(len(smooth_rl_vals_array[13]) / 5)]

smooth_rl_vals_ad_E_e4 = smooth_rl_vals_array[14][:int(len(smooth_rl_vals_array[14]) / 5)]
smooth_rl_vals_ad_E_nc = smooth_rl_vals_array[15][:int(len(smooth_rl_vals_array[15]) / 5)]

'''smooth_rl_vals_cu_A_x = smooth_rl_vals_array[16][:int(len(smooth_rl_vals_array[16]) / 5)]
smooth_rl_vals_cu_B_x = smooth_rl_vals_array[17][:int(len(smooth_rl_vals_array[17]) / 5)]
smooth_rl_vals_ad_A_x = smooth_rl_vals_array[18][:int(len(smooth_rl_vals_array[18]) / 5)]
smooth_rl_vals_ad_B_x = smooth_rl_vals_array[19][:int(len(smooth_rl_vals_array[19]) / 5)]'''




###########################################################
###########################################################




ave_result_cu_A = ave_result_array[0] 
ave_result_cu_B = ave_result_array[1] 

ave_result_cu_C_e4 = ave_result_array[2]  
ave_result_cu_C_nc = ave_result_array[3]

ave_result_cu_D_e4 = ave_result_array[4] 
ave_result_cu_D_nc = ave_result_array[5]  

ave_result_cu_E_e4 = ave_result_array[6] 
ave_result_cu_E_nc = ave_result_array[7]  

ave_result_ad_A = ave_result_array[8] 
ave_result_ad_B = ave_result_array[9]  

ave_result_ad_C_e4 = ave_result_array[10] 
ave_result_ad_C_nc = ave_result_array[11] 

ave_result_ad_D_e4 = ave_result_array[12] 
ave_result_ad_D_nc = ave_result_array[13] 

ave_result_ad_E_e4 = ave_result_array[14]
ave_result_ad_E_nc = ave_result_array[15]  

'''ave_result_cu_A_x = ave_result_array[16] 
ave_result_cu_B_x = ave_result_array[17] 
ave_result_ad_A_x = ave_result_array[18] 
ave_result_ad_B_x = ave_result_array[19] '''

In [ ]:
# split the dataset into training and testing sets
###############################################################################################################################
X_train_male, X_test_male, y_train_male, y_test_male = train_test_split(x_male, y_male, test_size=test_split, random_state=None)
X_train_female, X_test_female, y_train_female, y_test_female = train_test_split(x_female, y_female, test_size=test_split, random_state=None)

D_length=len(X_train_female)
E_length=len(X_train_male)


X_train_C= np.concatenate((X_train_male,X_train_female))
y_train_C=np.concatenate((y_train_male,y_train_female))

X_train_D=np.concatenate((X_train_male[:(math.floor(D_length/2))],X_train_female[:(math.floor(D_length/2))]))
y_train_D=np.concatenate((y_train_male[:(math.floor(D_length/2))],y_train_female[:(math.floor(D_length/2))]))

X_train_E= np.concatenate((X_train_male[:(math.floor(E_length/2))],X_train_female[:(math.floor(E_length/2))]))
y_train_E=np.concatenate((y_train_male[:(math.floor(E_length/2))],y_train_female[:(math.floor(E_length/2))]))
#########################################################################################################################################


model = RandomForestRegressor(n_estimators=800,min_samples_split=10,  min_samples_leaf=2, max_features='sqrt',max_depth=30)
Num_models=250
for iteration in range(Num_models):
    ######################################################################

    model.fit(X_train_male, y_train_male)
    # make predictions on the testing data
    y_pred_cn_A = model.predict(X_test_male)
    ensemble_predictions_cn_male.append(y_pred_cn_A)

    y_pred_ad1 = model.predict(x_ad_male)
    ensemble_predictions_ad_male.append(y_pred_ad1)

    ####################################################################
    #SHAP, uncomment to save SHAP data
    
    '''  explainer = shap.Explainer(model)

    shap_values_ml_cn = explainer(X_test_male)

    SHAPsMale_cn.append(shap_values_ml_cn)
   
    shap_values_ml_ad = explainer(x_ad_male)
    SHAPsMale_ad.append(shap_values_ml_cn)'''

    ####        ####    ####
    #Now non-APOE4  ####
    ####        ####    ####
        ########################################################################
    model.fit(X_train_female, y_train_female)
    
    # make predictions on the testing data
    y_pred_cn_B = model.predict(X_test_female)
    ensemble_predictions_cn_female.append(y_pred_cn_B)

    #####################
    ####AD
    #####################
    y_pred_ad2 = model.predict(x_ad_female)
    ensemble_predictions_ad_female.append(y_pred_ad2)
    ########################################################################
    
    #######SHAP, uncomment to save SHAP data

    '''   
    explainer = shap.Explainer(model)
    
    shap_values_fm_cn = explainer(X_test_female)
    SHAPsFemale_cn.append(shap_values_fm_cn)
    
    shap_values_fm_ad = explainer(x_ad_male)
    SHAPsFemale_ad.append(shap_values_fm_ad)
    '''

#############################################################################
    model.fit(X_train_C, y_train_C)

    # make predictions on the testing data
    y_pred_cn3 = model.predict(X_test_male)
    ensemble_predictions_cn_male3.append(y_pred_cn3)

    y_pred_ad3 = model.predict(x_ad_male)
    ensemble_predictions_ad_male3.append(y_pred_ad3)
   

    y_pred_cn3_non = model.predict(X_test_female)
    ensemble_predictions_cn_female3.append(y_pred_cn3_non)

    y_pred_ad3_non = model.predict(x_ad_female)
    ensemble_predictions_ad_female3.append(y_pred_ad3_non)
###############################################################################

    
    ####################
    #######SHAP,uncomment to save SHAP data
    ####################
    '''
    explainer = shap.Explainer(model)
    

    shap_values_ml_cn3= explainer(X_test_male)
    SHAPsMale_cn3.append(shap_values_ml_cn3)
    shap_values_ml_ad3= explainer(x_ad_male)
    SHAPsMale_ad3.append(shap_values_ml_ad3)


    shap_values_fm_cn3= explainer(X_test_female)
    SHAPsFemale_cn3.append(shap_values_fm_cn3)
    shap_values_fm_ad3  = explainer(x_ad_female)
    SHAPsFemale_ad3.append(shap_values_fm_ad3)
    '''

    #############################################################################
    model.fit(X_train_D, y_train_D)

    y_pred_cn_D = model.predict(X_test_female)
    ensemble_predictions_cn_D_fm.append(y_pred_cn_D)

    y_pred_ad_D = model.predict(x_ad_female)
    ensemble_predictions_ad_D_fm.append(y_pred_ad_D)
    #####################################################################################
    model.fit(X_train_E, y_train_E)

    y_pred_cn_E = model.predict(X_test_male)
    ensemble_predictions_cn_E_ml.append(y_pred_cn_E)

    y_pred_ad_E = model.predict(x_ad_male)
    ensemble_predictions_ad_E_ml.append(y_pred_ad_E)
    #####################################################################################
    
    ####################
    #######SHAP, uncomment if collecting SHAP data
    ####################
    '''
    explainer = shap.Explainer(model)
    
    
    shap_values_ml_cn4= explainer(X_test_male)
    SHAPsMale_cn4.append(shap_values_ml_cn4)
    shap_values_ml_ad4= explainer(x_ad_male)
    SHAPsMale_ad4.append(shap_values_ml_ad4)


    shap_values_fm_cn4= explainer(X_test_female)
    SHAPsFemale_cn4.append(shap_values_fm_cn4)
    shap_values_fm_ad4  = explainer(x_ad_female)
    SHAPsFemale_ad4.append(shap_values_fm_ad4)
    '''
    print(iteration)



ave_ensemble_prediction_cn_fm = np.mean(ensemble_predictions_cn_female, axis=0)
ave_ensemble_prediction_ad_fm = np.mean(ensemble_predictions_ad_female, axis=0)

ave_ensemble_prediction_cn_ml = np.mean(ensemble_predictions_cn_male, axis=0)
ave_ensemble_prediction_ad_ml = np.mean(ensemble_predictions_ad_male, axis=0)

ave_ensemble_prediction_cn_C_fm = np.mean(ensemble_predictions_cn_female3, axis=0)
ave_ensemble_prediction_ad_C_fm = np.mean(ensemble_predictions_ad_female3, axis=0)

ave_ensemble_prediction_cn_C_ml = np.mean(ensemble_predictions_cn_male3, axis=0)
ave_ensemble_prediction_ad_C_ml = np.mean(ensemble_predictions_ad_male3, axis=0)

ave_ensemble_prediction_cn_D_ml = np.mean(ensemble_predictions_cn_D_fm, axis=0)
ave_ensemble_prediction_ad_D_ml = np.mean(ensemble_predictions_ad_D_fm, axis=0)

ave_ensemble_prediction_cn_E_fm = np.mean(ensemble_predictions_cn_E_ml, axis=0)
ave_ensemble_prediction_ad_E_fm = np.mean(ensemble_predictions_ad_E_ml, axis=0)



Saving Data

In [151]:
import numpy as np

# Replace with your file path
file_path = 'C:/Users/woods/OneDrive/Documents/Brain Lab/brain age paper/New code/risk_factor_APOE4'

# Saving the NumPy arrays
np.savetxt(f'{file_path}/ave_results_cn1.csv', ave_result_cu_A, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/ave_results_ad1.csv', ave_result_ad_A, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/ave_results_cn2.csv', ave_result_cu_B, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/ave_results_ad2.csv', ave_result_ad_B, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/ave_results_cn3_APOE4.csv', ave_result_cu_C_e4, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/ave_results_ad3_APOE4.csv', ave_result_ad_C_e4, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/ave_results_cn3_non.csv', ave_result_cu_C_nc, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/ave_results_ad3_non.csv', ave_result_ad_C_nc, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/ave_results_cn4_APOE4.csv', ave_result_cu_D_e4, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/ave_results_ad4_APOE4.csv', ave_result_ad_D_e4, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/ave_results_cn4_nc.csv', ave_result_cu_D_nc, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/ave_results_ad4_nc.csv', ave_result_ad_D_nc, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/ave_results_cn5_APOE4.csv', ave_result_cu_E_e4, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/ave_results_ad5_APOE4.csv', ave_result_ad_E_e4, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/ave_results_cn5_nc.csv', ave_result_cu_E_nc, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/ave_results_ad5_nc.csv', ave_result_ad_E_nc, delimiter=',', fmt='%f')





np.savetxt(f'{file_path}/smooth_rl_vals_cn1.csv', smooth_rl_vals_cu_A, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/smooth_rl_vals_ad1.csv', smooth_rl_vals_ad_A, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/smooth_rl_vals_cn2.csv', smooth_rl_vals_cu_B, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/smooth_rl_vals_ad2.csv', smooth_rl_vals_ad_B, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/smooth_rl_vals_cn3_APOE4.csv', smooth_rl_vals_cu_C_e4, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/smooth_rl_vals_ad3_APOE4.csv', smooth_rl_vals_ad_C_e4, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/smooth_rl_vals_cn3_non.csv', smooth_rl_vals_cu_C_nc, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/smooth_rl_vals_ad3_non.csv', smooth_rl_vals_ad_C_nc, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/smooth_rl_vals_cn4_APOE4.csv', smooth_rl_vals_cu_D_e4, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/smooth_rl_vals_ad4_APOE4.csv', smooth_rl_vals_ad_D_e4, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/smooth_rl_vals_cn4_nc.csv', smooth_rl_vals_cu_D_nc, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/smooth_rl_vals_ad4_nc.csv', smooth_rl_vals_ad_D_nc, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/smooth_rl_vals_cn5_APOE4.csv', smooth_rl_vals_cu_E_e4, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/smooth_rl_vals_ad5_APOE4.csv', smooth_rl_vals_ad_E_e4, delimiter=',', fmt='%f')

np.savetxt(f'{file_path}/smooth_rl_vals_cn5_nc.csv', smooth_rl_vals_cu_E_nc, delimiter=',', fmt='%f')
np.savetxt(f'{file_path}/smooth_rl_vals_ad5_nc.csv', smooth_rl_vals_ad_E_nc, delimiter=',', fmt='%f')



Loading Data

In [ ]:
import numpy as np

# Replace with your file path
file_path = 'C:/Users/woods/OneDrive/Documents/Brain Lab/brain age paper/New code/risk_factor_APOE4'

# Loading the NumPy arrays
ave_result_cu_A = np.loadtxt(f'{file_path}/ave_results_cn1.csv', delimiter=',', dtype=int)
ave_result_ad_A = np.loadtxt(f'{file_path}/ave_results_ad1.csv', delimiter=',', dtype=int)

ave_result_cu_B = np.loadtxt(f'{file_path}/ave_results_cn2.csv', delimiter=',', dtype=int)
ave_result_ad_B = np.loadtxt(f'{file_path}/ave_results_ad2.csv', delimiter=',', dtype=int)

ave_result_cu_C_e4 = np.loadtxt(f'{file_path}/ave_results_cn3_APOE4.csv', delimiter=',', dtype=int)
ave_result_ad_C_e4 = np.loadtxt(f'{file_path}/ave_results_ad3_APOE4.csv', delimiter=',', dtype=int)

ave_result_cu_C_nc = np.loadtxt(f'{file_path}/ave_results_cn3_non.csv', delimiter=',', dtype=int)
ave_result_ad_C_nc = np.loadtxt(f'{file_path}/ave_results_ad3_non.csv', delimiter=',', dtype=int)

ave_result_cu_D_e4 = np.loadtxt(f'{file_path}/ave_results_cn4_APOE4.csv', delimiter=',', dtype=int)
ave_result_ad_D_e4 = np.loadtxt(f'{file_path}/ave_results_ad4_APOE4.csv', delimiter=',', dtype=int)

ave_result_cu_D_nc = np.loadtxt(f'{file_path}/ave_results_cn4_nc.csv', delimiter=',', dtype=int)
ave_result_ad_D_nc = np.loadtxt(f'{file_path}/ave_results_ad4_nc.csv', delimiter=',', dtype=int)

ave_result_cu_E_e4 = np.loadtxt(f'{file_path}/ave_results_cn5_APOE4.csv', delimiter=',', dtype=int)
ave_result_ad_E_e4 = np.loadtxt(f'{file_path}/ave_results_ad5_APOE4.csv', delimiter=',', dtype=int)

ave_result_cu_E_nc = np.loadtxt(f'{file_path}/ave_results_cn5_nc.csv', delimiter=',', dtype=int)
ave_result_ad_E_nc = np.loadtxt(f'{file_path}/ave_results_ad5_nc.csv', delimiter=',', dtype=int)

smooth_rl_vals_cu_A = np.loadtxt(f'{file_path}/smooth_rl_vals_cn1.csv', delimiter=',', dtype=int)
smooth_rl_vals_ad_A = np.loadtxt(f'{file_path}/smooth_rl_vals_ad1.csv', delimiter=',', dtype=int)

smooth_rl_vals_cu_B = np.loadtxt(f'{file_path}/smooth_rl_vals_cn2.csv', delimiter=',', dtype=int)
smooth_rl_vals_ad_B = np.loadtxt(f'{file_path}/smooth_rl_vals_ad2.csv', delimiter=',', dtype=int)

smooth_rl_vals_cu_C_e4 = np.loadtxt(f'{file_path}/smooth_rl_vals_cn3_APOE4.csv', delimiter=',', dtype=int)
smooth_rl_vals_ad_C_e4 = np.loadtxt(f'{file_path}/smooth_rl_vals_ad3_APOE4.csv', delimiter=',', dtype=int)

smooth_rl_vals_cu_C_nc = np.loadtxt(f'{file_path}/smooth_rl_vals_cn3_non.csv', delimiter=',', dtype=int)
smooth_rl_vals_ad_C_nc = np.loadtxt(f'{file_path}/smooth_rl_vals_ad3_non.csv', delimiter=',', dtype=int)

smooth_rl_vals_cu_D_e4 = np.loadtxt(f'{file_path}/smooth_rl_vals_cn4_APOE4.csv', delimiter=',', dtype=int)
smooth_rl_vals_ad_D_e4 = np.loadtxt(f'{file_path}/smooth_rl_vals_ad4_APOE4.csv', delimiter=',', dtype=int)

smooth_rl_vals_cu_D_nc = np.loadtxt(f'{file_path}/smooth_rl_vals_cn4_nc.csv', delimiter=',', dtype=int)
smooth_rl_vals_ad_D_nc = np.loadtxt(f'{file_path}/smooth_rl_vals_ad4_nc.csv', delimiter=',', dtype=int)

smooth_rl_vals_cu_E_e4 = np.loadtxt(f'{file_path}/smooth_rl_vals_cn5_APOE4.csv', delimiter=',', dtype=int)
smooth_rl_vals_ad_E_e4 = np.loadtxt(f'{file_path}/smooth_rl_vals_ad5_APOE4.csv', delimiter=',', dtype=int)

smooth_rl_vals_cu_E_nc = np.loadtxt(f'{file_path}/smooth_rl_vals_cn5_nc.csv', delimiter=',', dtype=int)
smooth_rl_vals_ad_E_nc = np.loadtxt(f'{file_path}/smooth_rl_vals_ad5_nc.csv', delimiter=',', dtype=int)


Creating Scatter Plots and Calculating ID Values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from scipy.integrate import quad
import numpy as np

#smooth_rl_vals_cu_A = np.array(smooth_rl_vals_cu_A)

def line_of_best_fit(x, slope, intercept):
    return slope * x + intercept

def calculate_mae(y_true, y_pred, slope, intercept):
    best_fit_vals = line_of_best_fit(y_true, slope, intercept)
    return mean_absolute_error(y_pred, best_fit_vals)

def integrand(x, a, b, c, d):
    return (c * x + d) - (a * x + b)

def plot_scatter_with_fit(y_true_cn, y_pred_cn, y_true_ad, y_pred_ad, x_label, y_label, group_label):
    fig, ax = plt.subplots()
    ax.scatter(y_true_cn, y_pred_cn, marker='o', label='CU',color='purple')
    ax.scatter(y_true_ad, y_pred_ad, marker='^', label='AD',color='green')
    ax.legend()
    ax.plot([55, top_age], [55, top_age], linestyle='--', color='black')
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_xlim(55, top_age)
    ax.set_ylim(55, 90)

    unique_y_true_cn = np.unique(y_true_cn)
    unique_y_true_ad = np.unique(y_true_ad)
    ax.plot(unique_y_true_cn, np.poly1d(np.polyfit(y_true_cn, y_pred_cn, 1))(unique_y_true_cn), color='purple', label='Line of Best Fit (CU)')
    ax.plot(unique_y_true_ad, np.poly1d(np.polyfit(y_true_ad, y_pred_ad, 1))(unique_y_true_ad), color='green', label='Line of Best Fit (AD)')
    plt.show()

    

def process_data(y_true_cn, y_pred_cn, y_true_ad, y_pred_ad, group_label):
    plot_scatter_with_fit(y_true_cn, y_pred_cn, y_true_ad, y_pred_ad, 'Chronological Age', 'Brain Estimate Age', group_label)
    
    coefficients_cn = np.polyfit(y_true_cn, y_pred_cn, 1)
    slope_cn, y_int_cn = coefficients_cn
    coefficients_ad = np.polyfit(y_true_ad, y_pred_ad, 1)
    slope_ad, y_int_ad = coefficients_ad

    print(f"Slope for CU {group_label}:", (slope_cn),"x+",y_int_cn)
    print(f"Slope for AD {group_label}:", (slope_ad),"x+",y_int_ad)

    diff_factor, _ = quad(integrand, 55, top_age, args=(slope_cn, y_int_cn, slope_ad, y_int_ad))
    print(f"The BAD for {group_label} is:", diff_factor/20)


    


    def calculate_std(y_true, y_pred, slope, intercept):
        # Calculate residuals
        residuals = y_pred - (slope * y_true + intercept)
        # Calculate standard deviation of residuals
        std_dev = np.std(residuals)
        return std_dev
    mae_cn = calculate_mae(y_true_cn, y_pred_cn, slope_cn, y_int_cn)
    mae_ad = calculate_mae(y_true_ad, y_pred_ad, slope_ad, y_int_ad)

    cn_sigma = calculate_std(y_true_cn, y_pred_cn, slope_cn, y_int_cn)
    ad_sigma = calculate_std(y_true_ad, y_pred_ad, slope_ad, y_int_ad)

    print(f"MAE for CU {group_label} line of best fit:", mae_cn)
    print(f"MAE for AD {group_label} line of best fit:", mae_ad)
    
    print(f"Standard deviation for CU {group_label} line of best fit:", cn_sigma)
    print(f"Standard deviation for AD {group_label} line of best fit:", ad_sigma)


    # Define the function for mu
    def mu_function_cn(slope,t,cn_int):#this find the "average", or point on the LOBF
        return slope * t + cn_int
    def mu_function_ad(slope,t,ad_int):#this find the "average", or point on the LOBF
        return slope * t + ad_int
        

        # Define the normal PDFs where mu is a function of t
    def normal_pdf_cn(x, t, sigma,slope,int):
            mu = mu_function_cn(slope,t,int)
            return norm.pdf(x, mu, sigma)

    def normal_pdf_ad(x, t, sigma,slope,int):
        mu = mu_function_ad(slope,t,int)
        return norm.pdf(x, mu, sigma)

    def diff_at_point(x, t, cn_slope, ad_slope, cn_int, ad_int, cn_sigma, ad_sigma):
        pdf_difference = abs(normal_pdf_cn(x, t, cn_sigma, cn_slope, cn_int) - normal_pdf_ad(x, t, ad_sigma, ad_slope, ad_int))
        return pdf_difference
    def integrate_diff_over_t(t):
        a,b= quad(diff_at_point, -np.inf, np.inf, args=(t, slope_cn, slope_ad, y_int_cn, y_int_ad, cn_sigma, ad_sigma))
        return a
    t_min = 55
    t_max = top_age

    integrated_difference1, error = quad(integrate_diff_over_t, 55, 59)

    integrated_difference2, error = quad(integrate_diff_over_t, 59, 63)
    integrated_difference3, error = quad(integrate_diff_over_t, 63, 67)
    integrated_difference4, error = quad(integrate_diff_over_t, 67, 71)
    integrated_difference5, error = quad(integrate_diff_over_t, 71,75)
    integrated_difference, error = quad(integrate_diff_over_t, 55, top_age)
    
    integrated_difference, error = quad(integrate_diff_over_t, t_min, t_max)
    
    print("ID value:",(integrated_difference)/40)

##################### Uncommment below sections for ID value of limited ranges. ##################
    '''
    print("IDV1:",integrated_difference1/8)
    print("IDV2:",integrated_difference2/8)
    print("IDV3:",integrated_difference3/8)
    print("IDV4:",integrated_difference4/8)
    print("IDV5:",integrated_difference5/8)
'''
############################################################################################################################################################
process_data(smooth_rl_vals_cu_A[smooth_rl_vals_cu_A<top_age] ,ave_result_cu_A[smooth_rl_vals_cu_A<top_age] ,               smooth_rl_vals_ad_A ,ave_result_ad_A,  'Model A APOE4')
process_data(smooth_rl_vals_cu_B[smooth_rl_vals_cu_B<top_age] ,ave_result_cu_B[smooth_rl_vals_cu_B<top_age],                smooth_rl_vals_ad_B ,ave_result_ad_B,  'Model B non')

process_data(smooth_rl_vals_cu_C_e4[smooth_rl_vals_cu_C_e4<top_age] ,ave_result_cu_C_e4[smooth_rl_vals_cu_C_e4<top_age],                smooth_rl_vals_ad_C_e4 ,ave_result_ad_C_e4,  'Model C APOE4')
process_data(smooth_rl_vals_cu_C_nc[smooth_rl_vals_cu_C_nc<top_age] ,ave_result_cu_C_nc[smooth_rl_vals_cu_C_nc<top_age],                smooth_rl_vals_ad_C_nc ,ave_result_ad_C_nc,  'Model C non')

process_data(smooth_rl_vals_cu_D_e4[smooth_rl_vals_cu_D_e4<top_age] ,ave_result_cu_D_e4[smooth_rl_vals_cu_D_e4<top_age],                smooth_rl_vals_ad_D_e4 ,ave_result_ad_D_e4,  'Model D APOE4')

process_data(smooth_rl_vals_cu_D_nc[smooth_rl_vals_cu_D_nc<top_age] ,ave_result_cu_D_nc[smooth_rl_vals_cu_D_nc<top_age],                smooth_rl_vals_ad_D_nc ,ave_result_ad_D_nc,  'Model D non')

process_data(smooth_rl_vals_cu_E_e4[smooth_rl_vals_cu_E_e4<top_age] ,ave_result_cu_E_e4[smooth_rl_vals_cu_E_e4<top_age],                smooth_rl_vals_ad_E_e4 ,ave_result_ad_E_e4,  'Model E APOE4')
process_data(smooth_rl_vals_cu_E_nc[smooth_rl_vals_cu_E_nc<top_age] ,ave_result_cu_E_nc[smooth_rl_vals_cu_E_nc<top_age],                smooth_rl_vals_ad_E_nc ,ave_result_ad_E_nc,  'Model E non')
##############################################################################################################################################################'''



